In [1]:
import pandas as pd
import numpy as np
import csv
import re

from unidecode import unidecode #para los acentos

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [2]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [26]:
df = pd.read_csv("../data/srape-mercadona_2024-03-08.csv")

,categoria,subcategoria,descripcion,unidades,precio
0,Agua y refrescos,Isotónico y energético,Bebida isotónica limón Aquarius,8 latas x 330 ml,"6,80 €6,24 € /pack"
1,Agua y refrescos,Isotónico y energético,Bebida isotónica limón Aquarius,8 latas x 330 ml,"6,80 €6,24 € /pack"
2,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Botella 1 L,"9,25 € /ud."
3,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Garrafa 3 L,"29,55 € /ud."
4,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Botella 1 L,"9,90 € /ud."
...,...,...,...,...,...
5030,Zumos,Tomate y otros sabores,Néctar de maracuyá Hacendado fruta de la pasió...,Brick 1 L,"0,81 € /ud."
5031,Zumos,Tomate y otros sabores,Bebida de mango Hacendado sin azúcares añadidos,Brick 1 L,"1,25 € /ud."
5032,Zumos,Tomate y otros sabores,Bebida de pomelo Hacendado sin azúcares añadidos,Brick 1 L,"1,25 € /ud."
5033,Zumos,Tomate y otros sabores,Zumo de manzana selección Hacendado,Brick 1 L,"1,50 € /ud."


# 1. EXPLORE

In [4]:
df.sample()

#categoria, subcategoria y descripcion - las dejamos como están. 

,categoria,subcategoria,descripcion,unidades,precio
4967,Postres y yogures,Yogures naturales y sabores,Yogur sabor plátano Hacendado,4 ud. x 125 g,"0,80 € /pack"


In [5]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 19 duplicate rows


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
# All collumns have values. 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5016 entries, 0 to 5034
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     5016 non-null   object
 1   subcategoria  5016 non-null   object
 2   descripcion   5016 non-null   object
 3   unidades      5016 non-null   object
 4   precio        5016 non-null   object
dtypes: object(5)
memory usage: 235.1+ KB


# 2. TRANFORM

##  - PRECIO

- No hay valores nulos

- El precio puede contener o 2 informaciones o 3:
    - El precio y a que hace referencia ese precio (pack)
    - El precio, precio si esta descontado y a que hace referencia ese precio(pack)

- Creamos una funcion que aplicamos a todos los elementos de la columna precio para formaterla. Aqui extraeremos los tres valores comentados antes. Cuando no haya descuento --> np.nan
    - price_info --> string
    - price_eu and price_disc_eu --> float
    
  **Note:** Interesante aqui el tema de apply(function) y luego el lambda. 
  
- Eliminamos la columna original

**NOTE:** En price_info, los valores son bastante consistentes siendo la mayoria de valoes (ud/pack). No obstante hay 14 valores que son de tipo ( xxx g) ej 500 g. Veremos a ver que hacemos. si los descartamos o como los tranformamos. No son una parte muy relevante del conjunto de datos ( 14 de 5035) pero tener en cuenta que hay que revisarlo: 


**Tranform Price column**

In [8]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        info: pos[0]: str 
        valores: pos[1]: float
        descuento: pos[2]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").split("/")
    
    # 2. Split by the "/" to extract the info, keep last part(always at the end)
    info = x[-1].replace(".", "") #(str)
    
    # 3. Splits by the "/" to extract the num values:
    num_values = x[0:-1][0].replace("," , ".").strip().split(" ")
    
    # 4. Extracts and formats price value:
    price = float(num_values[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(num_values) == 1 else float(num_values[1])][0]
    
    return [info, price, discount]

In [9]:
# Apply the function and get the element for each colum: 

"He decidodo que esto no aporta nada"
#df["price_info"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[2])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

df.sample()

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu
4501,Panadería y pastelería,Harina y preparado repostería,Masa filo Bella,Paquete 250 g,2.0,NaN


## - MARCA

Hay tres marcas principales que se van a repetir en casi la mitad de articulos, que son los nombres
de las marcas propias de mercadona: 


In [10]:
"""Funcion que te devuelve la marca si el texto la contiene"""
def marca(x):
    marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola"]
    
    """otras_marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola","Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares", "Mahou"] """
    
    
    return [i for i in marcas if i in x]

In [11]:
#Aplicamos la funcion, como realmente devuelve una lista, si esta vacia (no match) --> np.nan
df["marca"] = df["descripcion"].apply(marca).apply(lambda x: x[0] if x != [] else np.nan)

In [12]:
#Quiero ver cuantas filas no hemos identificado la marca 
df['marca'].isna().sum()

1984

Ver si queremos sacar el resto de marcas o no. Tener en cuenta que todo lo de la carne y tal creo que no 
pone que es acendado pero si que lo es, ver como vamos a hacer eso por si queremos comparar marcas blancas solo

Para intentar afinar un poco mas y sacar mas marcas, vamos a intentar lo siguiente: 

Me he dado cuenta de que las marcas estan siempre escritas La primera en  mayuscula.
La marca y la primera palabra de la descripcion, que suele hacer referencia al tipo de articulo
Nunca ponen la Marca lo primero, por lo que si esta en mayudculas sera o el primer elemento 
de la descripcion(hace referencia al producto) o la marca. 

In [13]:
#Dividimos las descripciones por palbras
lista = df["descripcion"].unique()
words_list = [ i.strip().split(" ") for i in lista]

In [402]:
# eliminamos la primera palabra de cada descirpcion( En mayusculas pero no es una marca)
palabras = []

for i in words_list:
    a = i[1:]
    for w in a:
        palabras.append(w)
        
#Nos queda una lista de palabras donde la mayoria de palabras estaran en minusculas, 
#sin embargo sabemos que las marcas tienen la primera letra en mayuscula: 

In [403]:
pattern = r"(\b[A-Z][a-zA-Z]*\b)" #Primera letra en mayucula
#Quitamos TODAS las palabras que no cumplan con el patron
clave = [word for word in palabras if re.match(pattern, word)]

In [404]:
"""Aqui podemos ver toas las palabras que empiezan por mayusculas y como se repiten, ya veremos
cuanto queremo pulir esto"""

#Counter(clave)

'Aqui podemos ver toas las palabras que empiezan por mayusculas y como se repiten, ya veremos\ncuanto queremo pulir esto'

In [405]:
# Esta es la lista de otras marcas que podemos incluir en la funcion: 

otras_marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola","Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares"]

### UNIDADES:

In [ ]:
Caso 1:
    
    ^[A-Z][a-zA-Z]+\s(?:\d+,)?\d+\s([K|k]g|g|L|ml)  --> 758 de /
    
Caso 2:
    (\d+)\s([a-zA-Z\s]+)\sx\s((?:\d+,)?\d+\s(kg|ml|g|L)) --> 149
    
Caso 3: Solo las unidades:
        ^(?:\d+,)?\d+\s+(kg|g|ml|L)$
        
Caso 4:
    (\w+)\s+(\d+)\s+(\w+)\s+(?:\(([\d,]+\s*(?:kg|g|ml|L))\)|([\d,]+\s*(?:kg|g|ml|L)))

In [25]:
len(unidades)

1396

In [42]:
df["unidades"] = df["unidades"].apply(unidecode)

In [44]:
patron = r'^[A-Z][a-zA-Z]+\s(?:\d+,)?\d+\s([K|k]g|g|L|ml)'
unidades = df["unidades"].unique()
for i in unidades:
    if " x " not in i:
        if re.match(patron, i):
            continue
        else:
            print(i.replace(".", ""))

Paquete 4 sobres (0,4 g)
Paquete 3 usos (0,38 g)
Caja 5 sobres (20 g)
Paquete 12 sobres (240 g)
Paquete 8 sobres (160 ml)
Caja 12 sobres (36 g)
Paquete 3 ud (270 g)
Paquete 3 ud (315 g)
Caja 18 placas (200 g)
Caja 12 placas (80 g)
Caja 20 placas (125 g)
Caja 50 sticks (350 g)
Caja 50 sobres (350 g)
Bote 850 pastillas (52 g)
Caja 60 sobres (60 g)
Bote 175 pastillas (9 g)
Caja 54 sobres (54 g)
Paquete 32 chicles (45 g)
Bote 70 chicles (100 g)
Paquete 12 chicles (31 g)
Paquete 25 chicles (100 g)
Paquete 12 ud (144 g)
Paquete 7 ud (105 g)
Paquete 10 ud (200 g)
Paquete 16 ud (200 g)
Paquete 5 ud (207,5 g)
Paquete 3 ud (141 g)
Paquete 5 ud (200 g)
50 g
25 g
Paquete 6 ud (135 g)
Paquete 6 ud (216 g)
35 g
120 g
40 g
60 g
Caja 22 ud (200 g)
Paquete 16 ud (180 g)
Caja 16 ud (216 g)
Caja 20 ud (205 g)
Paquete 20 ud (135 g)
Paquete 4 ud (75 g)
1 ud (120 g)
1 ud (100 g)
Caja 4 ud (360 g)
1 ud
Paquete 2 ud
Paquete 1 ud
Caja 30 ud
Paquete 100 ud
Paquete 10 ud
Caja 60 ud
Paquete 80 ud
Paquete 24 ud
Pa

In [ ]:
def descomp_unidades(x)
    
    # sabemos que se repite mucho el "aprox" y no nos aporta nada 
    x = x.replace("aprox.", "")

# Quitamos los acentos que pueda haber(Botellín no lo pilla el patron de regexx)
df["unidades"] = df["unidades"].apply(unidecode)

#extraemos las unidades
pattern = r'((?:\d+,)?\d+\s*(?:g|[Kk]g|L|ml))'
df["cantidad"] =  df["unidades"].str.extract(pattern)

In [406]:
# sabemos que se repite mucho el "aprox" y no nos aporta nada 
df["unidades"] = df["unidades"].str.replace("aprox.", "")

# Quitamos los acentos que pueda haber(Botellín no lo pilla el patron de regexx)
df["unidades"] = df["unidades"].apply(unidecode)

In [407]:
#extraemos las unidades
pattern = r'((?:\d+,)?\d+\s*(?:g|[Kk]g|L|ml))'
df["cantidad"] =  df["unidades"].str.extract(pattern)


In [408]:
# Extraemos el fomato
pattern = r"(\b[A-Z][a-zA-Z]*\b)" #Primera letra en mayucula
df["formato"] =  df["unidades"].str.extract(pattern)

In [409]:
df.sample()

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
4659,Panadería y pastelería,Velas y decoración,Kit decoración Hacendado,1 ud.,1.9,NaN,Hacendado,NaN,NaN


In [410]:
# Analizamos como ha quedado el data frame

## OJO LOS QUE CONTIENEN LA PALABRA EXURRIDO HAN SALIDO RAROS 
#CONSUM SOLO PONE EL ESCURRIDO ASI QUE VAMOS A QUERER SACAR EL ESCURRIDO 

In [411]:
df['cantidad'].isna().sum()

798

In [412]:
df['formato'].isna().sum()
#para ver porque han podido salir vacios

905

In [413]:
df[df['formato'].isna()]

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
0,Agua y refrescos,Isotónico y energético,Bebida isotónica limón Aquarius,8 latas x 330 ml,6.80,6.24,NaN,330 ml,NaN
24,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva, vinagre y sal Merry",15 monodosis x 10 ml,2.20,NaN,NaN,10 ml,NaN
91,"Aceite, especias y salsas","Mayonesa, ketchup y mostaza",Mayonesa en sobres individuales,12 sobres x 20 ml,1.25,NaN,NaN,20 ml,NaN
111,"Aceite, especias y salsas",Otras salsas,Tomate frito Hacendado,3 bricks x 400 g,1.40,NaN,Hacendado,400 g,NaN
112,"Aceite, especias y salsas",Otras salsas,Tomate frito Hacendado,3 mini bricks x 210 g,1.10,1.00,Hacendado,210 g,NaN
...,...,...,...,...,...,...,...,...,...
5010,Zumos,Melocotón y piña,Bebida de piña Hacendado sin azúcares añadidos,6 mini bricks x 200 ml,1.35,NaN,Hacendado,200 ml,NaN
5013,Zumos,Melocotón y piña,Zumo de piña selección Hacendado,3 mini bricks x 330 ml,2.25,NaN,Hacendado,330 ml,NaN
5019,Zumos,Naranja,Zumo pura naranja Hacendado,6 mini bricks x 200 ml,2.80,NaN,Hacendado,200 ml,NaN
5025,Zumos,Naranja,Bebida de naranja Hacendado sin azúcares añadidos,6 mini bricks x 200 ml,1.75,NaN,Hacendado,200 ml,NaN


In [414]:
# Inspeccionamos los que no han salido como queriamos: 

#Cuantos en la columna formato se han quedado vacios: 
vacios = df['formato'].isna().sum() # 934

#creamos un df con las filas donde formato se ha quedado vacio para inspeccionar mas a fondo
f_vacio = df[df['formato'].isna()]

f_vacio.sample()

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
5008,Zumos,Melocotón y piña,Zumo de piña y uva Hacendado,6 mini bricks x 200 ml,1.4,NaN,Hacendado,200 ml,NaN


In [415]:
#Vemos dos causas principales - las unidades que son 8 nnnn * no estan en mayuculas: 
f_vacio[f_vacio['unidades'].str.contains('x')].sample() #443 filas cumplen esta condicion

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
4819,Postres y yogures,Bífidus,Bífidus desnatado probiótico con kiwi Hacendad...,4 ud. x 125 g,1.3,NaN,Hacendado,125 g,NaN


In [416]:
#vamos a ver a el resto que le pasa: 
f_vacio[f_vacio['unidades'].str.contains('x') == False].sample() #491 filas cumplen esta condicion

# Vemos que en muchas simplemente no se especifica, o pone 1ud que no aporta nada 
# elemplo --> inidces = 4974 / 714 -- 160 g / 1 ud. (100 g)

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
4096,Maquillaje,Ojos,Máscara de pestañas Panoramic Deliplus negro,1 ud.,4.0,NaN,Deliplus,NaN,NaN


In [417]:
# los que NO tiene el formato de la x
sin_x = f_vacio[f_vacio['unidades'].str.contains('x') == False]

In [418]:
sin_x[sin_x["unidades"] == "1 ud."] #250 simplemente pone "1 ud."

,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
845,Bebé,Biberón y chupete,Biberón 150 ml tetina silicona flujo lento +0 ...,1 ud.,3.00,NaN,Deliplus,NaN,NaN
846,Bebé,Biberón y chupete,Biberón 360 ml tetina silicona tres posiciones...,1 ud.,3.50,NaN,Deliplus,NaN,NaN
1713,Charcutería y quesos,Embutido curado,Fuet espetec extra Hacendado,1 ud.,2.57,NaN,Hacendado,NaN,NaN
1724,Charcutería y quesos,Embutido curado,Longaniza clásica extra Hacendado,1 ud.,2.80,NaN,Hacendado,NaN,NaN
1725,Charcutería y quesos,Embutido curado,Longaniza de payés extra Hacendado,1 ud.,4.98,NaN,Hacendado,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4106,Maquillaje,Ojos,Sombra de ojos Long Lasting Deliplus 08,1 ud.,4.25,NaN,Deliplus,NaN,NaN
4107,Maquillaje,Ojos,Sombra de ojos Long Lasting Deliplus 12,1 ud.,3.00,NaN,Deliplus,NaN,NaN
4109,Maquillaje,Pinceles y brochas,Sacapuntas doble Deliplus lápiz fino y grueso,1 ud.,2.50,NaN,Deliplus,NaN,NaN
4110,Maquillaje,Pinceles y brochas,Esponja maquillaje Deliplus,1 ud.,3.00,NaN,Deliplus,NaN,NaN


In [419]:
#quitamos ruido -- lo que quiero es ver si en unidades pone lo mismo que en cantidad. 
# Quitamos el texto "1ud."
sin_x["unidades"].str.replace("1 ud.", "")
#quitamos los parentesis 
sin_x["unidades"].str.replace("(", "").str.replace(")", "")

# Ahora que lo hemos tranformado, vemos cuantos son iguales 
sin_x[sin_x['unidades'] == sin_x['cantidad']].sample() # 113 son iguales 


,categoria,subcategoria,descripcion,unidades,price_eu,price_disc_eu,marca,cantidad,formato
4685,Pizzas y platos preparados,Pizzas,Pizza barbacoa Hacendado,415 g,2.5,NaN,Hacendado,415 g,NaN


In [420]:
"""Hay 113 que si le quitamos "()" y "1 ud.", son iguales en unidades 
y cantidad es decir no pone nada del formato o podemos decir que es 1ud """ 

# Quiza podemos incluir en el patron reggex que estrae las unidades lo que 1ud. 

'Hay 113 que si le quitamos "()" y "1 ud.", son iguales en unidades \ny cantidad es decir no pone nada del formato o podemos decir que es 1ud '